<center>
    
# Adversarial Natural Language Processing

## 04 Adversarial Attack
### Riccardo Spolaor (864877)
    
</center>

## Preparation

### Setting the seeds for deterministic results


In [1]:
from utils.seed_setter import set_seed
set_seed()

## Adversarial Attack
### Selecting  the Reviews to Attack
Firstly an array of reviews is selected. For demonstation purposes just short reviews (500 characters max) are considered. The average review length is of about 1,279 characters. An *attack list* is then created. It contains reviews which are correctly classified from the Sentiment Analysis model that can be queried in a Black-Box system. Reviews which are already misclassified by the model are added to an *adversarial list*.

In [2]:
import pickle,os

with open(os.path.join('./pickle_data/train_test_data/test_data.pickle'), 'rb') as f:
    x_test, y_test = pickle.load(f)
f.close()

In [3]:
print('Average length of a review:')
sum(map(len,x_test))/len(x_test)

Average length of a review:


1279.084303030303

In [4]:
from utils.black_box import BlackBox
black_box = BlackBox()

In [5]:
all_preds = black_box.predict_all(x_test)

In [6]:
adversarial_list = []

attack_list = []

for sent, lab, pred in list(zip(x_test, y_test, all_preds)):
        if round(pred) == lab and len(sent) < 500:
            attack_list += [(sent,lab)]
        elif round(pred) != lab and len(sent) < 500:
            adversarial_list += [(sent,lab)]

In [7]:
print('Length of the Attack List:')
len(attack_list)

Length of the Attack List:


1610

In [8]:
print('Length of the list of already misclassified reviews:')
len(adversarial_list)

Length of the list of already misclassified reviews:


144

In [9]:
attack_list[:3]

[('This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.',
  0),
 ("The trailers get you to the movie, but the movie just wasn't worth my 8.50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they hadn't added this one to their resume's. To be honest, the book is better..",
  0),
 ('The message of this movie is "personality is more important than beauty". Jeanine Garofalo is supposed to be the "ugly duckling", but the funny thing is that she\'s not at all ugly (actually she\'s a lot more attractive than Uma Thurman, the friend who looks like a model).Now, would this movie work if the "ugly duckling" was really unattractive? When will Hollywood stop with this hypocrisy?In my opinion, despite the m

In [10]:
print('Accuracy for the attack list:')
black_box.evaluate([sent for sent, lab in attack_list], [lab for sent, lab in attack_list])

Accuracy for the attack list:
1610/1610 [==============================] - 4s 3ms/sample - loss: 0.0466 - acc: 1.0000


[0.0465955188139255, 1.0]

In [11]:
print('Accuracy for the adversarial list:')
black_box.evaluate([sent for sent, lab in adversarial_list], [lab for sent, lab in adversarial_list])

Accuracy for the adversarial list:
144/144 [==============================] - 0s 3ms/sample - loss: 2.2604 - acc: 0.0000e+00


[2.2603875266181097, 0.0]

### Genetic Optimization Attack Algorithm
The genetic attack algorithm's code is shown below. It creates a population of altered reviews each generation by substituting single words of the reviews of the previous population. It uses the *Words Distance Matrix* and the *Google One Billion Words Language Model* to select the best substitute candidates of a word at each perturbation. At the beginning the population of altered reviews is generated by perturbing the original one `__max_words` times. If a review of this population is an Adversarial Example it is returned from the algorithm. The next generation children reviews are created by random sampling with probability two parent reviews and by choosing each word from one or another at random. The `__max_children` best children are selected and each one of them is perturbed `__max_words` times. If one of these children reviews is an Adversarial Example it is then returned from the function. The process is repeated and if the algorithm doesn't create an Adversarial Example in `__max_gens` generations the attack fails. When a perturbation occurs stopwords, previously changed words and Named-Entites aren't considered for substitution.

In [15]:
import string
import re
import os
import numpy as np
import pickle
import licensed_scripts.lm_1b_eval as google_language_model_utils
import nltk
from utils.black_box import BlackBox
import time

nltk.download('maxent_ne_chunker', quiet = True)
nltk.download('words', quiet = True)

class Attacker(object):
    
    def __init__ (self):
        with open(os.path.join('./pickle_data/attack_utils/tokens_dictionary.pickle'), 'rb') as f:
            tokens_dictionary, inverse_tokens_dictionary = pickle.load(f)
            self.__tokens_dictionary = tokens_dictionary
            self.__inverse_tokens_dictionary = inverse_tokens_dictionary
        f.close()
        
        self.__black_box = BlackBox()
        self.__distance_matrix = np.load(os.path.join('./numpy_files/distance_matrix.npy'))
        self.__google_lm = google_language_model_utils.LM()
        self.__stopwords = nltk.corpus.stopwords.words('english')
        self.__sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        self.__tree_bank_word_tokenizer = nltk.tokenize.TreebankWordTokenizer()
        
        latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
        safe_characters = string.ascii_letters + string.digits + latin_similar + ' '
        safe_characters += "'"
        self.__safe_characters = safe_characters
        
        self.__max_words = 5 
        self.__top_neighbours = 8 
        self.__top_lm = 4
        self.__max_gens = 15
        self.__max_children = 2
        self.__max_pop_members = 5
        
        
    def __handle_contractions(self, sentence):
        sentence = self.__tree_bank_word_tokenizer.tokenize(sentence)
        return ' '.join(sentence)
    
    def __preprocess_sentence(self, sentence):
        chars = set(w for w in sentence)
        symbols = [c for c in chars if not c in self.__safe_characters]
        if(len(symbols) == 0):
            new_sentence = sentence[:]
        for symbol in symbols:
            new_sentence = sentence.replace(symbol, ' ' + symbol + ' ')
        new_sentence = self.__handle_contractions(new_sentence)
        return new_sentence
    
    def __split_review(self, review):
        sentences = self.__sentence_tokenizer.tokenize(review)
        labels = [None] * len(sentences)
        for index, sentence in enumerate(sentences):
            sentences[index] = self.__preprocess_sentence(sentence)
            sentences[index] = nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sentences[index])))
            labels[index] = [None] * len(sentences[index])
            for i, chunk in enumerate(sentences[index]):
                if hasattr(chunk, 'label'):
                    labels[index][i] = 1
                    sentences[index][i] = ' '.join(c[0] for c in chunk)
                else:
                    labels[index][i] = 0
                    sentences[index][i] = chunk[0]
        return sentences, labels
            
    def __most_similar(self, word, delta = 0.5, num_words = 20):
        try:
            index = self.__tokens_dictionary[word]
        except:
            return []

        if (index > self.__distance_matrix.shape[0]):
            return []

        dist_order = np.argsort(self.__distance_matrix[index,:])[1:num_words+1]
        dist_list = self.__distance_matrix[index][dist_order]

        mask = np.ones_like(dist_list)
        mask = np.where(dist_list < delta)
        return [self.__inverse_tokens_dictionary[index] for index in dist_order[mask]]         
    
    def __rejoin_review(self, sentences):
        new_sentences = sentences[:]
        for i, sent in enumerate (new_sentences):
            new_sentence = ' '.join(sent)
            new_sentence = re.sub(r' ([^A-Za-z0-9])', r'\1', new_sentence)
            new_sentences[i] = new_sentence
        return ' '.join(new_sentences)
    
    def __perturb(self, sentences, word_index, neighbours, y_target, changed_words_list = []):
        
        prefix = ' '.join(sentences[word_index[0]][ : word_index[1]])
        suffix = ' '.join(sentences[word_index[0]][word_index[1]+1 : -1])
        
        lm_preds = self.__google_lm.get_words_probs(
            prefix, 
            neighbours[ : min(self.__top_neighbours, len(neighbours))], 
            suffix
        )
        
        score_list = []
        for adv_w in np.argsort(lm_preds)[- min(self.__top_lm, len(lm_preds)) : ]: 
            adv_sentences = sentences[:]
            adv_splitted_text = adv_sentences[word_index[0]][:]
            adv_splitted_text[word_index[1]] = neighbours[adv_w]
            adv_sentences[word_index[0]] = adv_splitted_text
            adv_review = self.__rejoin_review(adv_sentences)
            score = self.__black_box.predict_sentiment(adv_review)
            score_list += [(adv_sentences, score, changed_words_list + [word_index])]
            
        adv_reviews_sorted =  sorted(score_list, key=lambda x: x[1])    
        
        
        if y_target == 0:
            return adv_reviews_sorted[0]
            
        else:
            return adv_reviews_sorted[-1]
        
        
    def __crossover(self, parent1, parent2):
        parent1_copy = parent1[0]
        parent2_copy = parent2[0]
        changed_word_list = []
        for i in range(len(parent1_copy)):
            for j in range(len(parent1_copy[i])):
                if np.random.uniform() < 0.5:
                    parent1_copy[i][j] = parent2_copy[i][j]
                    if (i,j) in parent2[2]:
                        changed_word_list += [(i,j)]
                elif (i,j) in parent2[2]:
                    changed_word_list += [(i,j)]
        score = self.__black_box.predict_sentiment(self.__rejoin_review(parent1_copy))
        return parent1_copy, score, changed_word_list
    
    def __get_neighbours_dictionary(self, sentences, labels):
        neighbours_dictionary = {}
        for sent_idx, sent in enumerate(sentences):
            for word_idx, word in enumerate(sent):
                if labels[sent_idx][word_idx] == 1:
                    neighbours_dictionary[(sent_idx, word_idx)] = []
                else:
                    neighbours_dictionary[(sent_idx, word_idx)] = self.__most_similar(
                        word = word.lower(), delta = 0.5, num_words = 50
                    )

        return neighbours_dictionary
    
    def __get_words_to_change(self, neighbours_dictionary, sentences, changed_word_list = []):
        neighbours_length = {key: len(value) for key, value in neighbours_dictionary.items()}

        for key in neighbours_length.keys():
            if sentences[key[0]][key[1]].lower() in self.__stopwords or key in changed_word_list:
                neighbours_length[key] = 0

        length_sum = sum(neighbours_length.values())

        if length_sum == 0:
            return None
        
        neighbours_length = {key: value/length_sum for key, value in neighbours_length.items()}

        probabilities = list(neighbours_length.values())

        random_choice_size = min(len(np.nonzero(probabilities)[0]), self.__max_words)
        
        return np.random.choice(
            len(neighbours_length.keys()), size = random_choice_size, replace = False, p = probabilities
        )
        
    
        
    def attack (self, x_orig, y_orig):
        
        attack_start_time = time.time()
        
        y_target = int(not y_orig)
        sentences, labels = self.__split_review(x_orig)
        
        neighbours_dictionary = self.__get_neighbours_dictionary(sentences, labels)
        words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences)
        
        if words_to_change is None:
            return None
        
        print('Original sentence: \n' + x_orig)
        print('Starting Score: {}; Original Label: {}; Target Label: {}\n'.format(
            self.__black_box.predict_sentiment(x_orig), y_orig, y_target
        ))
        
        print('Generating population... \n')
        
        generation_start_time = time.time()

        population = []
        for index in words_to_change:
            new_member = self.__perturb(
                sentences, 
                list(neighbours_dictionary.keys())[index], 
                neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                y_target
            )
            
            if round(new_member[1]) == y_target:
                print('ATTACK SUCCESS!!!\n')
                final_sentence = self.__rejoin_review(new_member[0])
                print('Final adversarial sentence:\n {} \n Score: {}\n'.format(
                    self.__rejoin_review(new_member[0]), new_member[1]
                ))
                print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
                return final_sentence
            
            population += [new_member]

        for i in range(self.__max_gens):

            if len(population) == 0:
                print('ATTACK FAILED...\n')
                print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
                return None
            
            print('Generation #{}: \n'.format(i+1))
            
            sorted_population =  sorted(population, key=lambda x: x[1])    


            if y_target == 0:
                best_attack = sorted_population[0]
                sorted_population = sorted_population[ : min(self.__max_pop_members, len(sorted_population))]

            else:
                best_attack = sorted_population[-1]
                sorted_population = sorted_population[- min(self.__max_pop_members, len(sorted_population)) : ]
                
            print('Best Adversarial:\n {} \n Score: {}\n'.format(self.__rejoin_review(best_attack[0]), best_attack[1]))
            
            print('%generation_time = {} seconds\n'.format(int(time.time() - generation_start_time)))
        

            if y_target == 1:
                pop_scores = np.array([score for review, score, _ in sorted_population])
            else:
                pop_scores = np.array([1 - score for review, score, _ in sorted_population])
                
            logits = np.exp(pop_scores / 0.3)
            selection_probabilities = logits / np.sum(logits)
                
            parent_list_1 = np.random.choice(
                len(sorted_population), size=len(sorted_population), p=selection_probabilities)
            parent_list_2 = np.random.choice(
                len(sorted_population), size=len(sorted_population), p=selection_probabilities)
            
            children = [self.__crossover(
                sorted_population[parent_list_1[i]], 
                sorted_population[parent_list_2[i]]
            ) for i in range(len(sorted_population))]
            
            sorted_children =  sorted(children, key=lambda x: x[1])    
            
            if y_target == 0:
                sorted_children = sorted_children[ : min(self.__max_children, len(sorted_children))]

            else:
                sorted_children = sorted_children[- min(self.__max_children, len(sorted_children)) : ]
            
            perturbated_children = []
            
            print('Regenerating population... \n')
            
            generation_start_time = time.time()
            
            for child in sorted_children:
                
                sentences = child[0]
                
                neighbours_dictionary = self.__get_neighbours_dictionary(sentences, labels)
                words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences, child[2])
                
                if words_to_change is None:
                    words_to_change = self.__get_words_to_change(neighbours_dictionary, sentences)
        
                if words_to_change is None:
                    continue

                for index in words_to_change:
                    new_member = self.__perturb(
                        sentences, 
                        list(neighbours_dictionary.keys())[index], 
                        neighbours_dictionary[list(neighbours_dictionary.keys())[index]], 
                        y_target,
                        child[1]
                    )
                    
                    
                    
                    if round(new_member[1]) == y_target:
                        print('ATTACK SUCCESS!!!\n')
                        final_sentence = self.__rejoin_review(new_member[0])
                        print('Final adversarial sentence:\n {} \n Score: {}\n'.format(
                            self.__rejoin_review(new_member[0]), new_member[1]
                        ))
                        print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
                        return final_sentence

                    perturbated_children += [new_member]

            population = [best_attack] + perturbated_children
        
        print('ATTACK FAILED...\n')
        print('%total_time = {} seconds\n\n'.format(int(time.time() - attack_start_time)))
        return None

In [16]:
attacker = Attacker()

LM vocab loading done


Recovering graph.


INFO:tensorflow:Recovering Graph google_language_model\graph-2016-09-10.pbtxt


Recovering checkpoint google_language_model\ckpt-*


### Testing
The algorithm process is tested on 25 reviews.

In [17]:
final_list = []

BATCH_SIZE = 25

for i in range(BATCH_SIZE):
    print('####################ATTACK {}/{}:####################'.format(i+1, BATCH_SIZE))
    new_review = attacker.attack(attack_list[i][0], attack_list[i][1])
    if new_review is None:
        new_review = attack_list[i][0]
    final_list += [(new_review, attack_list[i][1])]

####################ATTACK 1/25:####################
Original sentence: 
This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film made me wonder about Chuck film choices. He work on a real dog with this one.
Starting Score: 0.04802941903471947; Original Label: 0; Target Label: 1

Generating population... 

Generation #1: 

Best Adversarial:
 This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This film delivered me wonder about Chuck film choices. He work on a real dog with this one. 
 Score: 0.11358830332756042

%generation_time = 145 seconds

Regenerating population... 

Generation #2: 

Best Adversarial:
 This film could cure sleep disorders, thats how bad it is. The story dragged, and the bad guy is not that scary. You will not even see this one on TBS reruns. This f

Generation #2: 

Best Adversarial:
 Worst pile of drivel to date! Everyone involved with this productive should be ashamed of themselves. Not one single element of the movie was anything slightly like an original thoughts. A first grader telling you a story about nap time is more entertaining. 
 Score: 0.0014815827598795295

%generation_time = 284 seconds

Regenerating population... 

Generation #3: 

Best Adversarial:
 Worst pile of drivel to today! Everyone involved with this productive should be ashamed of themselves. Not one single element of the movie was anything slightly like an original thoughts. A premiere grader telling you a story about nap time is more hilarious. 
 Score: 0.0028019179590046406

%generation_time = 288 seconds

Regenerating population... 

Generation #4: 

Best Adversarial:
 Worst pile of drivel to today! Everyone involved with this productive should be ashamed of themselves. Not one single element of the cinema was anything slightly like an original thoughts

Regenerating population... 

Generation #3: 

Best Adversarial:
 What more can you ask for? A great screenplay based on one of the finest plays of the latter half of the 20th century, two fine emotional performances by Courtney and Finney, a realistic vision of war time london, a great aid blown. This film takes you on an emotional rollercoaster through humour, sadness, wasting and fulfillment. if you are in the theatre it is even more efficiency. This is a true 10 on the rating scale! 
 Score: 0.9874259233474731

%generation_time = 286 seconds

Regenerating population... 

Generation #4: 

Best Adversarial:
 What more can you ask for? A great screenplay based on one of the finest plays of the latter half of the 20th century, two fine emotional performances by Courtney and Finney, a realistic vision of war time london, a great aid blown. This film takes you on an emotional rollercoaster through humour, sadness, wasting and fulfillment. if you are in the theatre it is even more efficien

Original sentence: 
This is the very La Nouvelle Vague.One of the best films of the New Wave and I dare say one of the first ten ever made! Why? The atmosphere, the story,the actors (actress) are all brilliant. This is the theater, a fairy tale, the life, the film.Paris. Thank you Mr.Rivette.
Starting Score: 0.997246265411377; Original Label: 1; Target Label: 0

Generating population... 

Generation #1: 

Best Adversarial:
 This is the very La Nouvelle Vague. One of the best films of the New Wave and I dare say one of the first ten ever made! Why? The atmosphere, the stories, the actors( actress) are all brilliant. This is the theater, a fairy tale, the life, the film. Paris. Thank you Mr. Rivette. 
 Score: 0.9963240623474121

%generation_time = 142 seconds

Regenerating population... 

Generation #2: 

Best Adversarial:
 This is the very La Nouvelle Vague. One of the improved films of the New Wave and I dare say one of the prima ten ever made! Why? The atmosphere, the stories, the act

Regenerating population... 

Generation #4: 

Best Adversarial:
 My first Fassbinder was a magnificent experience. Film and alternative cinema( small hall, with uncomfortable seats; public had to wait while filmrolls were changed) were perfect match.There were many cliches used in the film, but Fassbinder presented them so wisely that I found them really amusing. Sound was similarly magnificent( sometimes backwards being louder than dialogue).Everything seemed to be in right putting. And I loved the way how after-war-time was presented. Real fun! 
 Score: 0.970556378364563

%generation_time = 286 seconds

Regenerating population... 

Generation #5: 

Best Adversarial:
 My initially Fassbinder was a magnificent experience. Film and alternative cinema( small hall, with uncomfortable seats; public had to wait while filmrolls were changed) were perfect match.There were many cliches used in the film, but Fassbinder presented them so wisely that I found them really amusing. Sound was similar

...

Regenerating population... 

Generation #5: 

Best Adversarial:
 I have to admit I've caught this one a few times on the USA Network. There's just something about the, well, merely stupidity of this flick which makes me want to watch it whenever it's on. yah, you're right about the sub-par acting, the plot which only an seven year old me like, etc. But I do n't help feeling sympathetic toward some of the actors. Then again, a few of these agents signed up for the even more grisly sequel. 
 Score: 0.045892566442489624

%generation_time = 286 seconds

Regenerating population... 

Generation #6: 

Best Adversarial:
 I have to admit I've caught this one a few times on the USA Network. There's just something about the, well, merely stupidity of this flick which makes me want to watch it whenever it's on. yah, you're right about the sub-par acting, the plot which only an seven year old me like, etc. But I do n't help feeling sympathetic toward some of the actors. Then again, a few of these a

Regenerating population... 

Generation #2: 

Best Adversarial:
 What can I tell? I know this movie from start to finish. It's hilarious. It's an forte link to my past and will change the way I view film in the future. Hypothetically speaking:) The down-fall? There's no Socrates Johnson! 
 Score: 0.9906947612762451

%generation_time = 285 seconds

Regenerating population... 

Generation #3: 

Best Adversarial:
 What can I tell? I know this movie from start to complete. It's hilarious. It's an forte bind to my past and will change the way I view film in the future. Hypothetically speaking:) The down-fall? There's no Socrates Johnson! 
 Score: 0.9702845811843872

%generation_time = 285 seconds

Regenerating population... 

Generation #4: 

Best Adversarial:
 What can I tell? I know this movie from start to complete. It's hilarious. It's an forte bind to my past and will alter the way I view film in the next. Hypothetically speaking:) The down-fall? There's no Socrates Johnson! 
 Score: 0

Generation #1: 

Best Adversarial:
 After looking John preform this one of a kind show, I had to share..... It was really something to watch a grown man portray himself as a child. I like the fact that with every character he`` became,`` you could picture what they looked like. It is more entertaining when you can understand the individual. `` Freak`` is what real`` stand up`` should be. John is REAL talent. 
 Score: 0.9087872505187988

%generation_time = 142 seconds

Regenerating population... 

Generation #2: 

Best Adversarial:
 After looking John preform this one of a kind show, I had to share..... It was really nothing to watch a grown man portray himself as a child. I like the fact that with every character he`` became,`` you could picture what they looked like. It is more entertaining when you can knowing the individual. `` Freak`` is what real`` stand up`` should be. John is REAL talent. 
 Score: 0.8346025943756104

%generation_time = 285 seconds

Regenerating population... 

G

Regenerating population... 

Generation #5: 

Best Adversarial:
 This is my favourites display. I thinks it is wholly magnificent. Thanks to David Chase for make this into my life. Season 11. The Sopranos: 5/52. 46 Long: 4. 5/53. Denial, Anger, Acceptance: 5/54. Meadowlands: 4/55. College:6. Pax Soprana: 5/57. Down Neck: 4. 5/58. The Legend of Tennessee Moltisanti: 5/59. Boca: 4. 5/510. A struck Is a struck: 3. 5/511. Nobody Knows nothing: 5/512. Isabella: 5/513. I Dream of Jeannie Cusamano: 5/5 
 Score: 0.6351128220558167

%generation_time = 286 seconds

Regenerating population... 

ATTACK SUCCESS!!!

Final adversarial sentence:
 This is my favourites display. I thinks it is utterly magnificent. Thanks to David Chase for produce this into my life. Season 11. The Sopranos: 5/52. 46 Long: 4. 5/53. Denial, Anger, Acceptance: 5/54. Meadowlands: 4/55. College:6. Pax Soprana: 5/57. Down Neck: 4. 5/58. The Legend of Tennessee Moltisanti: 5/59. Boca: 4. 5/510. A Hit Is a struck: 3. 5/511. Nob

Starting Score: 0.9529852867126465; Original Label: 1; Target Label: 0

Generating population... 

Generation #1: 

Best Adversarial:
 This film very succeed in the Film festival in Karlovy Vary, Czech rep. Logically. It' s based on a very good book, wonderful actors, good camera, the best director and ICELANDIC. It was probably the most black comedy I've seen. I really DO recommend it to you. 
 Score: 0.93238365650177

%generation_time = 142 seconds

Regenerating population... 

Generation #2: 

Best Adversarial:
 This film very succeed in the Film festival in Karlovy Vary, Czech rep. Logically. It' s based on a very decent book, wonderful actors, good camera, the higher director and ICELANDIC. It was probably the most black farce I've seen. I really DO recommend it to you. 
 Score: 0.7519599199295044

%generation_time = 284 seconds

Regenerating population... 

Generation #3: 

Best Adversarial:
 This film very succeed in the Film festival in Karlovy Vary, Czech rep. soundly. It' s b

In [18]:
print(final_list)

[('This movies could cure sleep disorders, dont how bad it is. The story dragged, and the naughty guy is not that scary. You will not even see this one on TBS reruns. This film delivered me wonder about Chuck film choices. He work on a real dog with this one.', 0), ("The trailers get you to the movie, but the movie just was n't worth my 8. 50.. it has some good effects, but the storyline.. yech, i like tommy lee jones and will smith as actors, they have both done some good films, but i wish they had n't added this one to their resume's. To be honest, the book is enhanced..", 0), ("The messenger of this movie is`` personality is more important than beauty``. Jeanine Garofalo is alleged to be the`` ugly duckling``, but the funny thing is that she's not at all ugly( actually she's a lot more attractive than Uma Thurman, the friend who looks like a model).Now, would this cinematic work if the`` foul duckling`` was really repellant? When will Hollywood stop with this hypocrisy? In my opinio

In [19]:
black_box.evaluate([sent for sent, lab in final_list], [lab for sent, lab in final_list])

25/25 [==============================] - 0s 7ms/sample - loss: 0.9364 - acc: 0.0000e+00


[0.9364231824874878, 0.0]